# 09.00-maj-causal-inference-direct-effects

In this notebook we revisit our causal inference model once again.
Using the online toolkit from Daggity, we can measure both the total effect and direct effect of

- SAT scores on acceptance
- Race on acceptance
- Gender on acceptance
<br></br>
- Total effect of SAT: $Y \sim SAT + GPA + Income$
- Direct effect of SAT: $Y \sim SAT + Gender + Race + School$
- Total effect of Gender/Race: $Y \sim Gender + GPA + SAT + Race + School$
- Direct effect of Gender/Race: $Y \sim Gender/Race$

In [63]:
import pandas as pd
import numpy as np
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from psis import *

import stan
import nest_asyncio
nest_asyncio.apply()

In [64]:
logistic_regression_code = """
/**
 * Logistic regression t-prior
 *
 * Priors:
 *     weights - student t
 *     intercept - student t
 */
data {
    int n;                        // number of data points
    int d;                        // explanatory variable dimension
    matrix[n, d] X;               // explanatory variable
    array[n] int <lower=0, upper=1> y;  // response variable
    int<lower=1> p_alpha_df;      // prior degrees of freedom for alpha
    real p_alpha_loc;             // prior location for alpha
    real p_alpha_scale;           // prior scale for alpha
    int<lower=1> p_beta_df;       // prior degrees of freedom for beta
    real p_beta_loc;     // prior location for beta
    real p_beta_scale;            // prior scale for beta

    int<lower=0> N_new;
    matrix[N_new, d] X_new;
}
parameters {
    real alpha;      // intercept
    vector[d] beta;  // explanatory variable weights
}
transformed parameters {
    vector[n] eta;  // linear predictor
    eta = alpha + X * beta;
}
model {
    alpha ~ student_t(p_alpha_df, p_alpha_loc, p_alpha_scale);
    beta ~ student_t(p_beta_df, p_beta_loc, p_beta_scale);
    y ~ bernoulli_logit(eta);
}
generated quantities {
    // calculate the log likelihood
    vector[n] log_lik;
    for (i in 1:n)
        log_lik[i] = bernoulli_logit_lpmf(y[i] | eta[i]);

    // generate values for Y tilde
    array[n] int<lower=0, upper=1> Y_tilde;
    Y_tilde = bernoulli_logit_rng(eta);

    // Make predictions
    vector[N_new] y_new;
    for (j in 1:N_new) {
        y_new[j] = bernoulli_logit_rng(alpha + X_new[j] * beta);
    }
}
"""



In [65]:
def run_model(data_dict):
    logistic_regression = stan.build(logistic_regression_code, data=data_dict)
    fit = logistic_regression.sample()
    return fit

def get_summary_table(fit, X_train_df, col=None):
    az_fit = az.from_pystan(fit)
    summary_table = az.summary(az_fit, var_names=['alpha', 'beta'])
    index_labels = ["intercept"] + list(X_train_df.columns)
    summary_table.index = index_labels
    if col:
        return summary_table.loc[col, :]
    else:
        return summary_table

In [66]:
df = pd.read_csv("../data/02-processed/normalized_data.csv")
df.dropna(inplace=True)
df

,SUBJID,SATMath,SATWriting,SATVerbal,GPA,state,gender,race,income,choice,accepted,school,numapply,habits,SAT
0,884230,-1.652536,-1.777725,-1.719744,-0.688423,AL,Female,Black,4.439333,4.0,1.0,2192.0,9,6,-1.918234
1,884232,-1.845287,-1.681540,-1.324951,-2.113622,AL,Male,Black,4.243038,1.0,1.0,2192.0,6,6,-1.809764
2,884233,-1.845287,-2.739570,-2.805425,-3.776353,AL,Male,Black,4.653213,4.0,0.0,2192.0,4,4,-2.749835
3,884247,-0.592404,-0.623510,-0.239270,0.974309,AL,Female,Black,4.096910,1.0,1.0,2192.0,9,8,-0.544284
4,884307,-0.977906,-0.334957,-0.930158,-0.688423,AL,Male,Black,4.096910,4.0,1.0,2192.0,9,7,-0.833536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91259,886635,-3.098170,-1.008248,-2.015839,0.261710,FL,Female,Two or more race/ethnicity,5.397940,1.0,1.0,1691.0,6,5,-2.279800
91260,886640,0.467728,0.530704,0.451618,0.261710,FL,Female,White,4.942008,1.0,1.0,1691.0,3,7,0.540414
91261,886642,-0.110526,1.684918,1.339902,0.974309,FL,Female,Two or more race/ethnicity,4.829304,1.0,1.0,1691.0,9,6,1.082762
91262,886648,-0.399653,-0.912064,-0.535365,-0.688423,FL,Male,White,4.829304,2.0,1.0,1691.0,6,7,-0.688910


## Total effect of SAT on Acceptance

In [67]:
total_sat_cols = ["SAT", "GPA", "income"]
total_gender_cols = ["gender", "GPA", "SAT", "race", "school"]
outcome = ["accepted"]

In [68]:
X = df[total_sat_cols].copy()
Y = df[outcome].copy()

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(
    X,
    Y,
    train_size=10_000,
    test_size=10,
    random_state=17)

# Convert dataframes to np arrays
X_train = X_train_df.copy().to_numpy()
X_test  = X_test_df.copy().to_numpy()
y_train = y_train_df.copy().astype(np.int8).to_numpy().flatten()
y_test = y_test_df.copy().astype(np.int8).to_numpy().flatten()

# Initialize data dictionary
n = X_train.shape[0]
d = X_train.shape[1]
degf = 1
data_dict = dict(
    n=n,                # num data points
    d=d,                # num features
    X=X_train,                # data matrix  
    y=y_train,                # response variable
    p_alpha_df=degf,   # prior deg freedom for alpha
    p_alpha_loc=0,      # prior location for alpha
    p_alpha_scale=5,  # prior scale for alpha
    p_beta_df=degf,    # prior deg freedom for beta
    p_beta_loc=0,       # prior location for beta
    p_beta_scale=5,    # prior scale for beta
    N_new=X_test.shape[0],
    X_new=X_test
)

In [69]:
fit = run_model(data_dict)

Building...



Building: found in cache, done.Messages from stanc:
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampli

In [70]:
get_summary_table(fit, X_train_df)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,1.416,0.239,0.939,1.856,0.006,0.004,1588.0,1765.0,1.0
SAT,-0.329,0.026,-0.376,-0.280,0.001,0.000,2540.0,2202.0,1.0
GPA,0.154,0.025,0.109,0.204,0.000,0.000,2484.0,2388.0,1.0
income,-0.120,0.052,-0.218,-0.020,0.001,0.001,1587.0,1707.0,1.0


The total effect of SAT scores on acceptance is $\exp(-0.372) \approx 0.69 \approx 31\%$  decrease in acceptance.

## Direct effect of SAT scores on Acceptance

In [71]:
direct_sat_cols = ["SAT", "gender", "race", "school"]
dummy_school = pd.get_dummies(df["school"], prefix="school")
dummy_race = pd.get_dummies(df["race"], prefix="", prefix_sep="")
dummy_gender = pd.get_dummies(df["gender"], prefix="", prefix_sep="")
X = pd.concat([df["SAT"], dummy_school, dummy_race, dummy_gender], axis=1).copy()
Y = df[outcome].copy()

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(
    X,
    Y,
    train_size=10_000,
    test_size=10,
    random_state=17)

# Convert dataframes to np arrays
X_train = X_train_df.copy().to_numpy()
X_test  = X_test_df.copy().to_numpy()
y_train = y_train_df.copy().astype(np.int8).to_numpy().flatten()
y_test = y_test_df.copy().astype(np.int8).to_numpy().flatten()

# Initialize data dictionary
n = X_train.shape[0]
d = X_train.shape[1]
degf = 7
data_dict = dict(
    n=n,                # num data points
    d=d,                # num features
    X=X_train,                # data matrix  
    y=y_train,                # response variable
    p_alpha_df=degf,   # prior deg freedom for alpha
    p_alpha_loc=0,      # prior location for alpha
    p_alpha_scale=5,  # prior scale for alpha
    p_beta_df=degf,    # prior deg freedom for beta
    p_beta_loc=0,       # prior location for beta
    p_beta_scale=5,    # prior scale for beta
    N_new=X_test.shape[0],
    X_new=X_test
)

In [72]:
fit = run_model(data_dict)

Building...



Building: found in cache, done.Messages from stanc:
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampli

In [75]:
get_summary_table(fit, X_train_df, ["SAT"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
SAT,-0.31,0.037,-0.381,-0.242,0.001,0.001,2668.0,3062.0,1.0


## Total effect of Gender/Race on Acceptance

In [76]:
X = pd.concat([df["SAT"], dummy_gender, dummy_race, dummy_school], axis=1)

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(
    X,
    Y,
    train_size=10_000,
    test_size=10,
    random_state=17)

# Convert dataframes to np arrays
X_train = X_train_df.copy().to_numpy()
X_test  = X_test_df.copy().to_numpy()
y_train = y_train_df.copy().astype(np.int8).to_numpy().flatten()
y_test = y_test_df.copy().astype(np.int8).to_numpy().flatten()

# Initialize data dictionary
n = X_train.shape[0]
d = X_train.shape[1]
degf = 7
data_dict = dict(
    n=n,                # num data points
    d=d,                # num features
    X=X_train,                # data matrix  
    y=y_train,                # response variable
    p_alpha_df=degf,   # prior deg freedom for alpha
    p_alpha_loc=0,      # prior location for alpha
    p_alpha_scale=5,  # prior scale for alpha
    p_beta_df=degf,    # prior deg freedom for beta
    p_beta_loc=0,       # prior location for beta
    p_beta_scale=5,    # prior scale for beta
    N_new=X_test.shape[0],
    X_new=X_test
)

In [77]:
%%capture
fit = run_model(data_dict)

In [78]:
get_summary_table(fit, X_train_df, list(dummy_gender) + list(dummy_race))

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Female,0.609,2.993,-4.991,6.294,0.088,0.062,1153.0,1752.0,1.00
Male,0.404,2.992,-5.187,6.065,0.088,0.062,1153.0,1703.0,1.00
American Indian,1.281,2.112,-2.947,5.196,0.079,0.056,711.0,995.0,1.01
Asian,-0.704,1.795,-4.130,2.578,0.075,0.053,565.0,932.0,1.01
Black,-0.084,1.792,-3.635,3.056,0.075,0.053,568.0,1009.0,1.01
Hispanic,0.041,1.793,-3.455,3.260,0.075,0.053,566.0,1008.0,1.01
Other,-0.175,1.800,-3.668,3.078,0.075,0.053,568.0,944.0,1.01
Two or more race/ethnicity,0.086,1.795,-3.331,3.374,0.075,0.053,565.0,969.0,1.01
White,0.356,1.793,-3.060,3.649,0.075,0.053,566.0,944.0,1.01


Both gender and race have a positive total effect on acceptance with the exception of being Asian and Hispanic. 
These correspond to a $\exp(-0.718) \approx 0.49 \approx 51\%$ and $\exp(-0.307) \approx 0.74 \approx 26\%$ reduction in the odds of getting accepted into their first choice school.
However the hdi for all variables includes negative and positive values and so the mean being centered between 0 to 1 for all variables could indicate a low degree of confidence for these estimated values. 

## Direct effect of Race on Accpetance

In [79]:
X = dummy_race

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(
    X,
    Y,
    train_size=10_000,
    test_size=10,
    random_state=17)

# Convert dataframes to np arrays
X_train = X_train_df.copy().to_numpy()
X_test  = X_test_df.copy().to_numpy()
y_train = y_train_df.copy().astype(np.int8).to_numpy().flatten()
y_test = y_test_df.copy().astype(np.int8).to_numpy().flatten()

# Initialize data dictionary
n = X_train.shape[0]
d = X_train.shape[1]
degf = 7
data_dict = dict(
    n=n,                # num data points
    d=d,                # num features
    X=X_train,                # data matrix  
    y=y_train,                # response variable
    p_alpha_df=degf,   # prior deg freedom for alpha
    p_alpha_loc=0,      # prior location for alpha
    p_alpha_scale=5,  # prior scale for alpha
    p_beta_df=degf,    # prior deg freedom for beta
    p_beta_loc=0,       # prior location for beta
    p_beta_scale=5,    # prior scale for beta
    N_new=X_test.shape[0],
    X_new=X_test
)

In [80]:
%%capture
fit = run_model(data_dict)

In [81]:
get_summary_table(fit, X_train_df)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,0.808,1.738,-2.339,4.346,0.097,0.087,331.0,291.0,1.02
American Indian,1.520,2.031,-2.514,5.326,0.105,0.074,383.0,527.0,1.01
Asian,-1.039,1.738,-4.566,2.097,0.097,0.087,331.0,293.0,1.02
Black,0.094,1.738,-3.686,3.006,0.097,0.083,332.0,299.0,1.02
Hispanic,-0.142,1.739,-4.054,2.649,0.097,0.086,332.0,286.0,1.02
Other,-0.370,1.745,-3.920,2.651,0.098,0.087,332.0,282.0,1.02
Two or more race/ethnicity,0.006,1.740,-3.543,3.149,0.097,0.085,332.0,290.0,1.02
White,0.337,1.738,-3.158,3.497,0.097,0.080,332.0,289.0,1.02


The direct effect of being Asian corresponds to a 56\% reduction in the odds of being accepted. 
Being hispanic now corresponds to only a $.002%$ reduction in the odds of being accepted.

## Direct effect of Gender on Acceptance

In [82]:
X = dummy_gender

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(
    X,
    Y,
    train_size=10_000,
    test_size=10,
    random_state=17)

# Convert dataframes to np arrays
X_train = X_train_df.copy().to_numpy()
X_test  = X_test_df.copy().to_numpy()
y_train = y_train_df.copy().astype(np.int8).to_numpy().flatten()
y_test = y_test_df.copy().astype(np.int8).to_numpy().flatten()

# Initialize data dictionary
n = X_train.shape[0]
d = X_train.shape[1]
degf = 7
data_dict = dict(
    n=n,                # num data points
    d=d,                # num features
    X=X_train,                # data matrix  
    y=y_train,                # response variable
    p_alpha_df=degf,   # prior deg freedom for alpha
    p_alpha_loc=0,      # prior location for alpha
    p_alpha_scale=5,  # prior scale for alpha
    p_beta_df=degf,    # prior deg freedom for beta
    p_beta_loc=0,       # prior location for beta
    p_beta_scale=5,    # prior scale for beta
    N_new=X_test.shape[0],
    X_new=X_test
)

In [83]:
%%capture
fit = run_model(data_dict)

In [84]:
get_summary_table(fit, X_train_df)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,0.443,2.946,-4.773,6.886,0.146,0.132,437.0,120.0,1.03
Female,0.468,2.946,-5.302,6.384,0.146,0.140,436.0,123.0,1.03
Male,0.332,2.946,-6.088,5.555,0.146,0.140,436.0,126.0,1.03


Both male and females have a po